# Miary
---
obliczenie miar zagregowanych (X) dla całej sieci

korzysta z plików .csv obliczonych w pliku `obliczenia.ipynb`

In [10]:
import pandas as pd
import numpy as np

#### parametrty do scenariuszy

In [3]:
B = 120 
t_ks =[30,40,50,60,70,80,90]
modes = ["PT","Car"]
aggs = ['mean','min','max','std','median', 'p_25', 'p_75']

In [ ]:
def calc_commute(trips,B,t_k,mode, folder = 'data//res//'):
    name = "_{}_{}_{}_no_filter".format(mode,str(B),str(t_k))
    print(name)
    print('A_jki')
    trips['A'] = B - trips.t - t_k
    trips.A[trips.A < 0] = 0
    trips.A[trips.A > 0] = trips.A + t_k
    print('A_ji_C')
    A_ji_C = trips[trips.A>0].groupby(by=['i','j'])['A'].agg(aggs)
    A_ji_C.columns = ['A_ji' + name + str(col)  for col in A_ji_C.columns]
    A_ji_C.to_csv(folder+'A_ji_C'+name+".csv")
    print('A_i_C')
    A_i_C = trips[trips.A>0].groupby(by=['i'])['A'].agg(aggs)
    A_i_C=A_i_C.reindex(rejony)
    A_i_C.columns = ['A_i_C' + name + str(col)  for col in A_i_C.columns]
    A_i_C.to_csv(folder+'A_i_C'+name+".csv")
    print('A_i_H')
    A_i_H = trips[(trips.i == trips.j) & (trips.A>0)].groupby(by=['i'])['A'].agg(aggs)
    A_i_H=A_i_H.reindex(rejony)
    A_i_H.columns = ['A_i_H' + name + str(col)  for col in A_i_H.columns]
    A_i_H.to_csv(folder+'A_i_H'+name+".csv") 

In [4]:
def readfull(ch,B,t_k,mode, agg, filtr = 'filter', col = 'A_i_', folder = 'data//res//'):
    """ otworz odpowiedni plik csv i załaduj do pandas"""
    name = "{}_{}_{}_{}_{}_".format(ch,mode,str(B),str(t_k), filtr)
    path = folder+col+name+".csv"  
    df = pd.read_csv(path, index_col=0)
    return df
df = readfull('C',120,60,'PT',None)


In [5]:
def X(ch,B,t_k, filtr = 'filter', col = 'A_i_'):
    """ otworz dla pliki csv i oblicz miatę X dla odpowiednich parametrów"""
    a = readone(ch,B,t_k,modes[0],aggs[0], filtr, col)
    b = readone(ch,B,t_k,modes[1],aggs[0], filtr, col)
    i = a.index.union(b.index)
    a = a.reindex(i).fillna(0)
    b = b.reindex(i).fillna(0)
    df =  pd.DataFrame((a - b)/(a  + b ))
    df.columns=['mean']
    for agg in aggs[1:]:
        a = readone(ch,B,t_k,modes[0],agg)
        b = readone(ch,B,t_k,modes[1],agg)
        i = a.index.union(b.index)
        a = a.reindex(i).fillna(0)
        b = b.reindex(i).fillna(0)
        df[agg] = (a - b)/(a  + b )
    return df

In [6]:
def readone(ch,B,t_k,mode, agg, filtr = 'filter', col = 'A_i_', folder = 'data//res//'):
    """ otworz odpowiedni plik csv i załaduj do pandas"""
    name = "{}_{}_{}_{}_{}_".format(ch,mode,str(B),str(t_k), filtr)
    
    path = folder+col+name+".csv"  
    df = pd.read_csv(path, index_col=0)
    if col == 'WA_i_':
        col = 'WA_ji_'
        return df[col+name[2:]+agg]
    return df[col+name+agg]


In [9]:
X("C",Bs[1],t_ks[3])

,mean,min,max,std,median,p_25,p_75
i,,,,,,,
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,0.013737,-0.001263,0.088420,0.693384,0.008131,0.006788,0.011094
4,0.015131,-0.000477,0.087960,0.670418,0.011740,-0.001622,0.014631
5,0.008331,0.001695,0.013751,0.789208,0.011574,0.002934,0.011762
6,0.020884,0.007201,0.088323,0.570621,0.021016,0.015031,0.021535
7,0.004579,-0.004378,0.016110,0.466880,0.004770,-0.000474,0.007895
8,-0.003814,0.001723,-0.010352,-0.485747,-0.003853,0.000106,-0.007120
9,0.006395,0.000424,0.087577,0.549493,0.000178,-0.000955,0.005330
10,0.010096,0.001892,0.017901,1.000000,0.010040,0.006893,0.013601


In [ ]:

folder = 'data//res//'
for ch in ["C","H"]:
    for B in Bs:
        for t_k in t_ks:
            if B>t_k:
                for filtr in ['no_filter','filter']:
                    for col in ['A_i_','WA_i_']:
                        name = "{}_{}_{}_{}".format(ch,str(B),str(t_k), filtr)
                        path = folder+'X_'+name+".csv" 
                        try:
                            X(ch,B,t_k,filtr, col).to_csv(path, sep = ";")  
                        except:
                            print(name)

----